In [5]:
import nbimporter
import item2_copy

### backend

In [22]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend=backend, shots=800,  seed_transpiler=seed, seed_simulator=seed)

### optimizer

In [23]:
from qiskit.algorithms.optimizers import L_BFGS_B

bfgs_optimizer = L_BFGS_B(maxiter=60)

In [24]:
import numpy as np
np.random.seed(999999)
target_distr = np.random.rand(2)
# We now convert the random vector into a valid probability vector
target_distr /= sum(target_distr)

In [25]:
import numpy as np
from qiskit.circuit.library import EfficientSU2

num_qubits = 2
entanglement = 'full'
energies = np.zeros(2)

ansatz = EfficientSU2(num_qubits=num_qubits, entanglement=entanglement)

In [12]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

params = np.random.rand(3)
print(get_var_form(params))

ret = bfgs_optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)
print(ret)
print(get_var_form(ret[0]))

     ┌────────────────────────────┐┌─┐
  q: ┤ U3(0.71981,0.081255,0.653) ├┤M├
     └────────────────────────────┘└╥┘
c: 1/═══════════════════════════════╩═
                                    0 


C:\Users\hadar\AppData\Local\Temp\ipykernel_20228\2317275169.py:6: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  qc.u3(params[0], params[1], params[2], qr[0])


NameError: name 'objective_function' is not defined

In [13]:
from qiskit import Aer, transpile, assemble
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0])
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    t_qc = transpile(qc, backend)
    qobj = assemble(t_qc, shots=NUM_SHOTS)
    result = backend.run(qobj).result()
    
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    
    return cost

In [14]:
from collections.abc import Sequence

import numpy as np

class ExpvalCost:
    """
    Create a cost function that gives the expectation value of an input Hamiltonian.

    This cost function is useful for a range of problems including VQE and QAOA.
    """

    def __init__(
        self,
        ansatz,
        hamiltonian,
        device,
    ):
        self.ansatz = ansatz
        
        coeffs, observables = hamiltonian.terms()

        self.hamiltonian = hamiltonian

        self._optimize = optimize

        self.qnodes = qml.map(
            ansatz, observables, device, interface=interface, diff_method=diff_method, **kwargs
        )

        if self._optimize:
            obs_groupings, coeffs_groupings = qml.grouping.group_observables(observables, coeffs)
            d = device[0] if self._multiple_devices else device
            w = d.wires.tolist()

            @qml.qnode(device, interface=interface, diff_method=diff_method, **kwargs)
            def circuit(*qnode_args, obs, **qnode_kwargs):
                """Converting ansatz into a full circuit including measurements"""
                ansatz(*qnode_args, wires=w, **qnode_kwargs)
                return [qml.expval(o) for o in obs]

            def cost_fn(*qnode_args, **qnode_kwargs):
                """Combine results from grouped QNode executions with grouped coefficients"""
                if device.shot_vector:
                    shots_batch = 0

                    for i in device.shot_vector:
                        shots_batch += i[1]

                    total = [0] * shots_batch

                    for o, c in zip(obs_groupings, coeffs_groupings):
                        res = circuit(*qnode_args, obs=o, **qnode_kwargs)

                        for i, batch_res in enumerate(res):
                            total[i] += sum(batch_res * c)
                else:
                    total = 0
                    for o, c in zip(obs_groupings, coeffs_groupings):
                        res = circuit(*qnode_args, obs=o, **qnode_kwargs)
                        total += sum([r * c_ for r, c_ in zip(res, c)])
                return total

            self.cost_fn = cost_fn

        else:
            self.cost_fn = qml.dot(coeffs, self.qnodes)


    def __call__(self, *params, state_idx):
        return self.cost_fn(*params, state_idx)


In [15]:
#Define in this box the parameters for the anzats
import numpy as np

#you may define here the requsted parameters as you'd like
thetas = np.array([0,0,0,0,np.pi/2,np.pi/2,np.pi/2,np.pi/2])
phis = np.array([np.pi,np.pi,np.pi,np.pi])

#you may define here the requested number of iterations D1 and D2 as described in Fig.1 of Subspace-search variational quantum
#eigensolver for excited states. 
D1 = 1
D2 = 1

qc = item2_copy.anzats_circ(thetas,phis,D1,D2)
qc.draw()

░ ┌───────┐┌─────────┐          ░ ┌───────┐»
q_0: ──────────────────────░─┤ Rx(0) ├┤ Rz(π/2) ├─■────────░─┤ Rx(0) ├»
                           ░ ├───────┤├─────────┤ │        ░ ├───────┤»
q_1: ──────────────────────░─┤ Rx(0) ├┤ Rz(π/2) ├─■──■─────░─┤ Rx(0) ├»
     ┌───────┐┌───────┐    ░ ├───────┤├─────────┤    │     ░ ├───────┤»
q_2: ┤ Rx(π) ├┤ Rz(π) ├─■──░─┤ Rx(0) ├┤ Rz(π/2) ├────■──■──░─┤ Rx(0) ├»
     ├───────┤├───────┤ │  ░ ├───────┤├─────────┤       │  ░ ├───────┤»
q_3: ┤ Rx(π) ├┤ Rz(π) ├─■──░─┤ Rx(0) ├┤ Rz(π/2) ├───────■──░─┤ Rx(0) ├»
     └───────┘└───────┘    ░ └───────┘└─────────┘          ░ └───────┘»
c: 4/═════════════════════════════════════════════════════════════════»
                                                                      »
«     ┌─────────┐
«q_0: ┤ Rz(π/2) ├
«     ├─────────┤
«q_1: ┤ Rz(π/2) ├
«     ├─────────┤
«q_2: ┤ Rz(π/2) ├
«     ├─────────┤
«q_3: ┤ Rz(π/2) ├
«     └─────────┘
«c: 4/═══════════
«

In [26]:
def get_k_basis(k, n):
    full_basis = np.identity(n)
    return full_basis[:k]

print(np.identity(4))
print(get_k_basis(2,4))

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [27]:
from qiskit import assemble, Aer
import math

def get_cost_function(params, basis_of_sub_space, D1, D2, hamiltonian):
    cost = 0
    for state_vector in basis_of_sub_space:
        quatnum_circut = item2_copy.anzats_circ(theta[0:8], theta[8:], D1, D2)
        quatnum_circut.initialize(state_vector)
        print(quatnum_circut.draw())
        
        qobj = assemble(quatnum_circut)
        state = backend.run(qobj).result().get_statevector()
        print(state)
        
        dagger_state = np.matrix(state)
        state = dagger_state.getH()
        state = np.array(state)
        dagger_state = np.array(dagger_state)
        print("--- state vectors now: ---")
        print(state)
        print(dagger_state)
        print("--- state vectors ends. ---")
        
        product = np.matmul(Ham, state)
        product = np.matmul(dagger_state, product)

        cost += product
        return cost

In [28]:
theta = np.zeros(12)
basis = get_k_basis(4,16)
target_func = get_cost_function(theta, basis, 1, 1, np.identity(16))
print("target func:")
print(target_func)

                           ░ ┌───────┐┌───────┐          ░ ┌───────┐┌───────┐»
q_0: ──────────────────────░─┤ Rx(0) ├┤ Rz(0) ├─■────────░─┤ Rx(0) ├┤ Rz(0) ├»
                           ░ ├───────┤├───────┤ │        ░ ├───────┤├───────┤»
q_1: ──────────────────────░─┤ Rx(0) ├┤ Rz(0) ├─■──■─────░─┤ Rx(0) ├┤ Rz(0) ├»
     ┌───────┐┌───────┐    ░ ├───────┤├───────┤    │     ░ ├───────┤├───────┤»
q_2: ┤ Rx(0) ├┤ Rz(0) ├─■──░─┤ Rx(0) ├┤ Rz(0) ├────■──■──░─┤ Rx(0) ├┤ Rz(0) ├»
     ├───────┤├───────┤ │  ░ ├───────┤├───────┤       │  ░ ├───────┤├───────┤»
q_3: ┤ Rx(0) ├┤ Rz(0) ├─■──░─┤ Rx(0) ├┤ Rz(0) ├───────■──░─┤ Rx(0) ├┤ Rz(0) ├»
     └───────┘└───────┘    ░ └───────┘└───────┘          ░ └───────┘└───────┘»
c: 4/════════════════════════════════════════════════════════════════════════»
                                                                             »
«     ┌──────────────────────────────────────────────┐
«q_0: ┤0                                             ├
«     │              

QiskitError: 'No statevector for experiment "None"'

In [4]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    print(sys.path)

from item2 import item2

ImportError: cannot import name 'item2' from 'item2' (unknown location)